# Chương 9: Không gian vector (I)
File enwiki-20150112-400-r10-105752.txt.bz2 là file nén dạng bzip2 của 105752 file text được lấy mẫu ngẫu nhiên (tỷ lệ 1/10) từ các bài báo trên Wikipedia có trên 400 từ. Các bài báo trên Wikipedia được lấy vào ngày 12 tháng 1 năm 2015. Sử dụng dữ liệu file này làm corpus để học các vector thể hiện ý nghĩa của các từ. Trong nửa đầu của chương 9, bạn được yêu cầu trích xuất các context của các từ, trích xuất đặc trưng, và dùng phương pháp PCA để giảm bớt số chiều của dữ liệu. Nửa sau của chương 9 yêu cầu bạn tính độ tương tự của các từ sử dụng các word vectors đã học từ corpus.

Chú ý, bài 83 yêu cầu 7GB memory. Trong trường hợp lượng memory của bạn không đủ, bạn cần có các phương pháp xử lý thích hợp hoặc sử dụng sample 1/100 của dữ liệu trong file enwiki-20150112-400-r10-105752.txt.bz2.

In [3]:
%cd /content/drive/My Drive/github_repos/practice_100_nlp_exercises/

/content/drive/My Drive/github_repos/practice_100_nlp_exercises


# Download data

In [0]:
!wget http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r10-105752.txt.bz2

--2020-04-18 15:55:04--  http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r10-105752.txt.bz2
Resolving www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)... 130.34.192.83
Connecting to www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)|130.34.192.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 221903910 (212M) [application/x-bzip2]
Saving to: ‘enwiki-20150112-400-r10-105752.txt.bz2’

enwiki-20150112-400 100%[===================>] 211.62M  11.2MB/s    in 19s     

2020-04-18 15:55:24 (11.1 MB/s) - ‘enwiki-20150112-400-r10-105752.txt.bz2’ saved [221903910/221903910]



In [0]:
!bzip2 -d enwiki-20150112-400-r10-105752.txt.bz2

# 80. Tiền xử lý dữ liệu
Sử dụng khoảng trắng là ký tự ngăn cách để tokenize các từ trong các câu. Phương pháp này có nhược điểm là các từ thu được sẽ còn các ký tự đặc biệt như dấu câu, hoặc dấu ngoặc. Vì thế sau khi tokenize các từ trong corpus, tiến hành các xử lý sau đây.

* Xoá các ký tự đặc biệt xuất hiện ở đầu và cuối các từ: .,!?;:()[]'"
* Xoá các từ chỉ gồm ký tự trắng

Sau khi tiền xử lý dữ liệu, lưu file dữ liệu gồm danh sách các từ cách nhau bởi khoảng trắng.


In [0]:
!head enwiki-20150112-400-r10-105752.txt

Anarchism

Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions, but that several authors have defined as more specific institutions based on non-hierarchical free associations. Anarchism holds the state to be undesirable, unnecessary, or harmful. While anti-statism is central, anarchism entails opposing authority or hierarchical organisation in the conduct of human relations, including, but not limited to, the state system.
As a subtle and anti-dogmatic philosophy, anarchism draws on many currents of thought and strategy. Anarchism does not offer a fixed body of doctrine from a single particular world view, instead fluxing and flowing as a philosophy. There are many types and traditions of anarchism, not all of which are mutually exclusive. Anarchist schools of thought can differ fundamentally, supporting anything from extreme individualism to complete collectivism. Strains of anarchism have often been divided int

In [0]:
with open('enwiki-20150112-400-r10-105752.txt', 'r') as f:
    lines = f.readlines()

len(lines)

2875326

In [0]:
import re

with open('enwiki_tokenized.txt', 'w') as f:
    for line in lines:
        line = line.strip()

        if line == '':
            continue

        # Remove special characters
        line = re.sub(r'[\.,!\?;:\(\)\[\]\'\"]', '', line)

        # Split by space
        line = line.split(' ')

        # Remove all empty elements
        line = list(filter(lambda a: a != '', line)) # Maybe use filter is faster than other methods

        # Join list by space
        line = ' '.join(line)

        # Write to file
        f.write(line + '\n')

In [0]:
!head 'enwiki_tokenized.txt'

Anarchism
Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions but that several authors have defined as more specific institutions based on non-hierarchical free associations Anarchism holds the state to be undesirable unnecessary or harmful While anti-statism is central anarchism entails opposing authority or hierarchical organisation in the conduct of human relations including but not limited to the state system
As a subtle and anti-dogmatic philosophy anarchism draws on many currents of thought and strategy Anarchism does not offer a fixed body of doctrine from a single particular world view instead fluxing and flowing as a philosophy There are many types and traditions of anarchism not all of which are mutually exclusive Anarchist schools of thought can differ fundamentally supporting anything from extreme individualism to complete collectivism Strains of anarchism have often been divided into the categories of

In [0]:
# Count number of lines (Linux)
!wc -l 'enwiki_tokenized.txt'

2557968 enwiki_tokenized.txt


In [0]:
# Count number of words (Linux)
!wc -w 'enwiki_tokenized.txt'

120307584 enwiki_tokenized.txt


# 81. Xử lý tên các nước tạo thành từ các compound words
Trong tiếng Anh, nhiều từ cạnh nhau có thể tạo thành một từ có ý nghĩa. Ví dụ, hợp chủng quốc Hoa Kỳ là "United States", vương quốc Anh là "United Kingdom". Nếu chỉ dùng các "United", "State", hay "Kingdom" như các từ riêng lẻ, ý nghĩa của các từ này sẽ nhập nhằng. Vì thế trong khi tiền xử lý dữ liệu, ta cần xác định các từ ghép này. Đoán nhận các từ ghép là một bài toán khó, nên ở đây ta chỉ đoán nhận các từ ghép là tên của các nước.

Trước hết, download danh sách tên của các nước trên Internet. Dùng danh sách tên các nước này để đoán nhận các từ ghép là tên nước trong dữ liệu sử dụng ở bài 80, sau đó biến đổi các ký tự spaces thành ký tự underscore (_) để nối các từ thành phần. Ví dụ "United States" sẽ trở thành "United_States", "Isle of Man" sẽ trở thành "Isle_of_Man."

In [0]:
# Download list country names from github
!wget -O country_names.txt https://raw.githubusercontent.com/datasets/country-list/master/data.csv

--2020-04-18 16:58:46--  https://raw.githubusercontent.com/datasets/country-list/master/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3870 (3.8K) [text/plain]
Saving to: ‘country_names.txt’

country_names.txt   100%[===================>]   3.78K  --.-KB/s    in 0s      

2020-04-18 16:58:46 (13.0 MB/s) - ‘country_names.txt’ saved [3870/3870]



In [0]:
# Get names only
import pandas as pd

country_name_df = pd.read_csv('country_names.txt')

country_name_df['Name'].to_csv('country_names.txt', index=False, header=False)

In [0]:
!head country_names.txt

Afghanistan
Åland Islands
Albania
Algeria
American Samoa
Andorra
Angola
Anguilla
Antarctica
Antigua and Barbuda


In [0]:
# Get country names which have more than 2 words
list_country_names = [re.sub('[,\"\n\'\(\)\.]', '', s) for s in open('country_names.txt', 'r').readlines()]

country_names_more_than_2_words = [s for s in list_country_names if len(s.split()) > 1]

In [0]:
# Create compound words
def insert_underscore(matchobj):
    return '' + matchobj.group(0).replace(' ', '_')

content = open('enwiki_tokenized.txt', 'r').read()

content = re.sub("(%s)" % "|".join(country_names_more_than_2_words), insert_underscore, content)

with open('enwiki_tokenized_compound_words.txt', 'w') as f:
    f.write(content)

In [0]:
# To display word has _
!grep -oh "\w*_\w*" enwiki_tokenized_compound_words.txt

# 82. Trích xuất context
Sử dụng corpus được tạo ra trong bài tập 81, trích xuất context của tất cả các từ xuất hiện trong corpus. Context c của mỗi từ t trong dữ liệu sẽ cặp với từ t và xuất ra theo định dạng: các thông tin cách nhau bởi ký tự tab. Context của mỗi từ t được định nghĩa như sau:

* Trích xuất các từ ở trước và sau của t với kích thước cửa sổ là d (chú ý context words của t sẽ không bao gồm bản thân của từ t)
* Với mỗi từ t, kích thước của context (window size) d sẽ được chọn ngẫu nhiên trong tập {1, 2, 3, 4, 5}.

In [0]:
# Read file
with open('enwiki_tokenized_compound_words.txt', 'r') as f:
    lines = f.readlines()

from random import randint # for random window size


with open('word_context.txt', 'w') as f:
    for line in lines:
        list_word = line.strip().split()

        n_word = len(list_word)

        for idx,word in enumerate(list_word):
            window_size = randint(1, 5)

            list_context = list_word[max(0, idx - window_size):idx] + list_word[
                                                                                min(idx+1, n_word): min(idx+1+window_size, n_word)]

            for context in list_context:
                f.write(word + '\t' + context + '\n')

In [0]:
!head word_context.txt

Anarchism	is
Anarchism	a
is	Anarchism
is	a
is	political
is	philosophy
is	that
is	advocates
a	Anarchism
a	is


# 83. Tính tần xuất xuất hiện của từ/context
Sử dụng đầu ra của bài 82, tính phân bố xuất hiện và các hằng số sau:

* f(t,c): là số lần đồng xuất hiện của từ t và context word c.
* f(t,*): số lần xuất hiện của từ t.
* f(*,c): số lần xuất hiện của context word c.
* N: Tổng số lần xuất hiện của từ và các context word (hằng số).

In [8]:
# N
!wc -l word_context.txt

689513516 word_context.txt


In [9]:
int(689513516/1000)

689513

In [0]:
# Create new file with fewer lines
!head --lines 689513 word_context.txt > word_context_fewer.txt

In [0]:
with open('word_context_fewer.txt') as f:
    lines = f.readlines()

token_context_dict = dict()
token_dict = dict()
context_dict = dict()

for line in lines:
    t = line.split()[0]
    c = line.split()[1]

    token_context_dict[(t, c)] = token_context_dict.get((t, c), 0) + 1
    token_dict[t] = token_dict.get(t, 0) + 1
    context_dict[c] = token_dict.get(c, 0) + 1

N = len(lines)

## f(t, c)

In [133]:
def freq_word_context(t, c):
    if (t, c) in token_context_dict:
        return token_context_dict[(t, c)]

    return None

freq_word_context('the', 'a')

369

## f(t, *)

In [134]:
def freq_word(t):
    if t in token_dict:
        return token_dict[t]

    return None

freq_word('the')

45554

## f(*, c)

In [135]:
def freq_context(c):
    if c in context_dict:
        return context_dict[c]
    
    return None

freq_context('as')

6582

# 84. Tạo Matrix của các từ và context words
Sử dụng đầu ra của bài 83, tạo ma trận word/context X. Các thành phần X_tc trong ma trận X được định nghĩa như sau.

* Nếu f(t,c) >= 10, X_tc = PPMI(t,c) = $max\{log\frac{N \times f(t, c)}{f(t, *) \times f(*, c)}; 0\}$
* Nếu f(t,c) < 10, X_tc = 0.

Ở đây PPMI(t,c) là ký hiệu của Positive Pointwise Mutual Information. Chú ý vì kích thước ma trận X là rất lớn, nên lưu tất cả các giá trị của ma trận vào bộ nhớ là không thể. Bạn có thể sử dụng kỹ thuật lưu trữ ma trận thưa với chú ý rằng, phần lớn giá trị của các phần tử trong X bằng 0.

In [0]:
from math import log2

In [0]:
def PPMI(freq_tc, freq_t, freq_c):
    return max(log2((N*freq_tc)/(freq_t * freq_c)), 0)

In [122]:
with open('word_context_fewer.txt') as f:
    lines = f.readlines()

token_context_dict = dict()
token_dict = dict()
context_dict = dict()

word2index = dict()
index2word = dict()
num_words = 0

for line in lines:
    t = line.split()[0]
    c = line.split()[1]

    if t not in word2index:
        word2index[t] = num_words
        index2word[num_words] = t
        num_words += 1

    if c not in word2index:
        word2index[c] = num_words
        index2word[num_words] = c
        num_words += 1

    token_context_dict[(t, c)] = token_context_dict.get((t, c), 0) + 1
    token_dict[t] = token_dict.get(t, 0) + 1
    context_dict[c] = token_dict.get(c, 0) + 1

N = len(lines)
len(token_context_dict), len(token_dict), len(context_dict), N

(415325, 17834, 17834, 689513)

In [125]:
# for creating sparce matrices
row_idxs = []
col_idxs = []

ppmi_data_values = []

for (t, c) in token_context_dict:
    row_idxs.append(word2index[t])
    col_idxs.append(word2index[c])

    if token_context_dict[(t, c)] >= 10:
        ppmi_data_values.append(PPMI(token_context_dict[(t, c)], token_dict[t], context_dict[c]))
    else:
        ppmi_data_values.append(0.0)
 
print('Done') 

Done


In [130]:
# Convert to matrix
from scipy import sparse
import pandas as pd

df = pd.DataFrame({'t':row_idxs, 'c':col_idxs, 'ppmi': ppmi_data_values})

mat = sparse.csr_matrix((df['ppmi'],(df['t'], df['c'])))
print(mat.todense()) 

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.60648799 ... 0.         0.         0.        ]
 [0.         0.64284531 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [136]:
mat.shape

(17834, 17834)

# 85. Sử dụng PCA để giảm số chiều dữ liệu
Sử dụng thuật toán PCA cho ma trận thu được trong bài tập 84 để giảm số chiều dữ liệu sao cho các word vector thu được có số chiều là 300.

In [0]:
from sklearn.decomposition import PCA
pca = PCA(n_components=300)

mat_300d = pca.fit_transform(mat.todense())

In [144]:
mat_300d.shape

(17834, 300)

In [145]:
mat_300d

array([[-1.50577205e-02, -1.70661802e-02,  1.79528800e-02, ...,
        -3.72588168e-17, -1.07605405e-16,  5.40471129e-17],
       [ 7.96198012e+00,  4.86308175e-01,  3.17799155e+00, ...,
         4.81220215e-19,  4.12945403e-20, -6.61935347e-20],
       [ 2.99528994e+00,  2.89267310e+00,  2.68034780e+00, ...,
         4.81220215e-19,  4.12945403e-20, -6.61935347e-20],
       ...,
       [-1.50577205e-02, -1.70661802e-02,  1.79528800e-02, ...,
         1.93027207e-19, -1.62697909e-19,  1.28616686e-19],
       [-1.50577205e-02, -1.70661802e-02,  1.79528800e-02, ...,
         3.46129729e-19, -1.62732284e-19,  1.37305292e-19],
       [-1.50577205e-02, -1.70661802e-02,  1.79528800e-02, ...,
        -6.78940046e-20,  8.21477993e-20, -9.71873191e-20]])

# 86. Hiển thị word vectors
Đọc vào các word vectors trong bài tập 85, hiển thị vector cho từ "United States". Chú ý là từ "United States" trong corpus đã được biến đổi thành "United_States."

In [146]:
mat_300d[word2index['United_States']]

array([ 3.10648949e-01,  8.01205950e-01, -1.10597821e+00, -3.07412464e-01,
        1.56520044e-01,  3.23654477e-01, -2.90397470e-01,  2.97768678e-01,
       -3.38878671e-01,  1.49367257e-02,  8.23161983e-01,  1.27988720e-01,
        7.24988472e-01,  2.12401035e-01,  6.11074711e-02, -1.01522096e-01,
        4.50441677e-02, -9.62439500e-02, -2.64443152e-01,  2.98412698e-03,
       -5.39854864e-01,  1.63756396e-01, -2.41104075e-01,  1.07173268e-02,
       -2.02592221e-01, -5.39475140e-02,  4.51857327e-02, -1.09607022e-01,
        1.39373126e-01, -2.79826719e-02,  7.15326449e-02,  9.82820166e-02,
       -6.76113331e-02, -6.27206402e-03, -7.72541973e-02,  6.41102030e-03,
       -1.84360254e-01, -3.89053754e-03,  5.02196080e-02,  2.65717492e-01,
        1.01337015e-02, -4.05799498e-02, -1.77803613e-02, -2.00559624e-01,
       -1.49293041e-02, -6.49061851e-02, -2.46318012e-02,  9.45431768e-02,
        1.80092958e-03,  1.78220236e-03,  3.57425194e-04,  1.58870470e-03,
       -1.53781165e-03, -

# 87. Tính word similarity
Sử dụng word vectors thu được trong bài tập 85, tính cosine similarity cho hai từ "United States" và "U.S." Chú ý là từ "U.S." trong corpus được lưu trữ là "U.S"

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

def ww_sim(word_vec1, word_vec2):
    return cosine_similarity(word_vec1, word_vec2).flatten()

In [152]:
united_states_wv = mat_300d[word2index['United_States']].reshape(1, -1)
us_wv = mat_300d[word2index['US']].reshape(1, -1)

ww_sim(united_states_wv, us_wv)

array([0.97428314])